In [1]:
# MM-SEFDM: STANDALONE TEST SCRIPT
# Test trained models without access to training code
# Only requires saved files: lut.pkl, phase2_weights.h5, stats.pkl

import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model

# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ============ CONFIGURATION ============
# **EDIT THESE PARAMETERS TO RUN YOUR TEST**

# Path to saved files
SAVED_FILES_PATH = "./MM_SEFDM_GitHub"

# Test parameters - CUSTOMIZE THESE
ALPHA_TEST_VALUES = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]  # Bandwidth compression values
SNR_TEST_VALUES = np.arange(0, 36, 5)  # SNR values in dB
NUM_TEST_SYMBOLS = 1000  # Number of symbols to test per point
CHANNEL_TYPE = 'rayleigh'  # 'awgn' or 'rayleigh'

# Plot settings
SAVE_PLOT = True
PLOT_FILENAME = "test_results.png"

print("="*70)
print("MM-SEFDM: STANDALONE TESTING")
print("="*70)
print(f"Loading files from: {SAVED_FILES_PATH}")
print(f"Test Parameters:")
print(f"  Alpha values: {ALPHA_TEST_VALUES}")
print(f"  SNR range: {SNR_TEST_VALUES[0]} to {SNR_TEST_VALUES[-1]} dB")
print(f"  Symbols per test: {NUM_TEST_SYMBOLS}")
print(f"  Channel: {CHANNEL_TYPE}")
print("="*70)

# ============ Helper Functions ============

def simple_progress(iterable, desc="Progress", leave=True):
    """Simple progress indicator"""
    total = len(iterable) if hasattr(iterable, '__len__') else None
    for i, item in enumerate(iterable):
        if total and ((i + 1) % max(1, total // 10) == 0 or (i + 1) == total):
            print(f"\r{desc}: {i+1}/{total}", end='', flush=True)
        yield item
    if leave:
        print()

# ============ Signal Processing Functions ============

def sefdm_modulate(freq_signal, alpha):
    """SEFDM modulation with bandwidth compression alpha"""
    N = len(freq_signal)
    time_signal = np.zeros(N, dtype=complex)
    for n in range(N):
        for k in range(N):
            time_signal[n] += freq_signal[k] * np.exp(2j * np.pi * k * alpha * n / N)
    time_signal /= np.sqrt(N)
    return time_signal

def generate_rayleigh_channel(N, channel_model='flat'):
    """Generate Rayleigh fading channel"""
    if channel_model == 'flat':
        h = (np.random.randn() + 1j * np.random.randn()) / np.sqrt(2)
        return h * np.ones(N, dtype=complex)
    return (np.random.randn(N) + 1j * np.random.randn(N)) / np.sqrt(2)

def add_awgn(signal, snr_db):
    """Add AWGN noise to signal"""
    if snr_db == np.inf:
        return signal
    snr_linear = 10 ** (snr_db / 10)
    signal_power = np.mean(np.abs(signal)**2)
    noise_power = signal_power / snr_linear
    return signal + np.sqrt(noise_power / 2) * (np.random.randn(*signal.shape) +
                                                  1j * np.random.randn(*signal.shape))

# ============ Custom Layers (Required for Model Loading) ============

class PositionalEncoding(layers.Layer):
    """Trainable positional encoding"""
    def __init__(self, d_model, max_len=10, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model
        self.max_len = max_len

    def build(self, input_shape):
        self.pos_embedding = self.add_weight(
            name='pos_embedding',
            shape=(self.max_len, self.d_model),
            initializer='uniform',
            trainable=True
        )
        super().build(input_shape)

    def call(self, x):
        batch_size = tf.shape(x)[0]
        seq_len = tf.shape(x)[1]
        positions = self.pos_embedding[:seq_len, :]
        positions = tf.expand_dims(positions, 0)
        positions = tf.tile(positions, [batch_size, 1, 1])
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update({"d_model": self.d_model, "max_len": self.max_len})
        return config

class ChannelAdaptiveGating(layers.Layer):
    """Channel-adaptive gating mechanism"""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def build(self, input_shape):
        feature_dim = input_shape[0][-1]
        self.gate_net = keras.Sequential([
            layers.Dense(feature_dim * 2, activation='relu'),
            layers.Dense(feature_dim, activation='sigmoid')
        ], name='gate_network')
        super().build(input_shape)

    def call(self, inputs):
        features, channel_conditions = inputs
        gates = self.gate_net(channel_conditions)
        if len(features.shape) == 3:
            gates = tf.expand_dims(gates, 1)
        return features * gates

    def get_config(self):
        return super().get_config()

class TransformerBlock(layers.Layer):
    """Transformer block with multi-head attention"""
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.dropout_rate = dropout

    def build(self, input_shape):
        self.att = layers.MultiHeadAttention(
            num_heads=self.num_heads,
            key_dim=self.d_model // self.num_heads,
            dropout=self.dropout_rate
        )
        self.ffn = keras.Sequential([
            layers.Dense(self.ff_dim, activation='gelu'),
            layers.Dropout(self.dropout_rate),
            layers.Dense(self.d_model),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(self.dropout_rate)
        self.dropout2 = layers.Dropout(self.dropout_rate)
        super().build(input_shape)

    def call(self, x, training=False):
        attn_output = self.att(x, x, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "d_model": self.d_model,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "dropout": self.dropout_rate
        })
        return config

# ============ Model Architecture ============

def build_advanced_cnn_transformer_hybrid(input_shape, num_classes,
                                         d_model=64, num_heads=4, num_layers=2,
                                         ff_dim=128, dropout=0.15):
    """Build the model architecture"""
    signal_input = layers.Input(shape=input_shape, name='signal_input')
    alpha_input = layers.Input(shape=(1,), name='alpha_input')
    snr_input = layers.Input(shape=(1,), name='snr_input')
    channel_type_input = layers.Input(shape=(1,), name='channel_type_input')

    # CNN with residual connections
    conv1 = layers.Conv1D(32, 3, padding='same', activation='relu', name='conv1_k3')(signal_input)
    conv1 = layers.LayerNormalization(name='conv1_ln')(conv1)
    conv1 = layers.Dropout(dropout, name='conv1_dropout')(conv1)

    conv2 = layers.Conv1D(48, 5, padding='same', activation='relu', name='conv2_k5')(conv1)
    conv2 = layers.LayerNormalization(name='conv2_ln')(conv2)
    conv2 = layers.Dropout(dropout, name='conv2_dropout')(conv2)

    conv3 = layers.Conv1D(64, 7, padding='same', activation='relu', name='conv3_k7')(conv2)
    conv3 = layers.LayerNormalization(name='conv3_ln')(conv3)

    residual = layers.Conv1D(64, 1, name='residual_proj')(signal_input)
    cnn_features = layers.Add(name='residual_add')([conv3, residual])
    cnn_features = layers.Dropout(dropout, name='conv_final_dropout')(cnn_features)

    # Channel embeddings
    alpha_embed = layers.Dense(64, activation='relu')(alpha_input)
    alpha_embed = layers.LayerNormalization()(alpha_embed)
    alpha_embed = layers.Dense(64, activation='relu')(alpha_embed)
    alpha_embed = layers.LayerNormalization()(alpha_embed)
    alpha_embed = layers.Dense(d_model, activation='tanh')(alpha_embed)

    snr_embed = layers.Dense(64, activation='relu')(snr_input)
    snr_embed = layers.LayerNormalization()(snr_embed)
    snr_embed = layers.Dense(d_model, activation='tanh')(snr_embed)

    channel_type_embed = layers.Dense(32, activation='relu')(channel_type_input)
    channel_type_embed = layers.Dense(d_model//2, activation='tanh')(channel_type_embed)

    channel_conditions = layers.Concatenate()([alpha_embed, snr_embed, channel_type_embed])
    channel_conditions = layers.Dense(d_model*2, activation='gelu')(channel_conditions)
    channel_conditions = layers.Dropout(dropout)(channel_conditions)
    channel_conditions = layers.Dense(d_model, activation='tanh')(channel_conditions)

    # Channel-adaptive gating
    cnn_features_gated = ChannelAdaptiveGating()([cnn_features, channel_conditions])

    # Positional encoding + Transformer
    x = PositionalEncoding(d_model, max_len=input_shape[0])(cnn_features_gated)

    for i in range(num_layers):
        x = TransformerBlock(d_model, num_heads, ff_dim, dropout)(x)

    # Dual pooling
    global_avg = layers.GlobalAveragePooling1D()(x)
    global_max = layers.GlobalMaxPooling1D()(x)
    pooled = layers.Concatenate()([global_avg, global_max])

    # Fusion with channel conditions
    combined = layers.Concatenate()([pooled, channel_conditions])

    # Classification head
    x = layers.Dense(256, activation='gelu')(combined)
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(dropout*1.5)(x)

    x = layers.Dense(128, activation='gelu')(x)
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(dropout)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(
        inputs=[signal_input, alpha_input, snr_input, channel_type_input],
        outputs=outputs,
        name='Advanced_CNN_Transformer_Hybrid'
    )

    return model

# ============ Testing Functions ============

def transmit_receive_grouped_system(labels, lut, N_total, groups, alpha, snr_db,
                                    channel_type='rayleigh', rayleigh_model='flat'):
    """Transmit and receive for grouped system"""
    group_size = N_total // groups
    tx_time_total = np.zeros(N_total, dtype=complex)

    for g, label in enumerate(labels):
        tx_freq = lut[label]['freq_signal']
        tx_time = sefdm_modulate(tx_freq, alpha)
        tx_time_total[g*group_size:(g+1)*group_size] = tx_time

    if channel_type == 'awgn':
        channel_total = np.ones(N_total, dtype=complex)
    else:
        channel_total = np.zeros(N_total, dtype=complex)
        for g in range(groups):
            h = generate_rayleigh_channel(group_size, rayleigh_model)
            channel_total[g*group_size:(g+1)*group_size] = h

    rx_time_total = tx_time_total * channel_total
    rx_time_total = add_awgn(rx_time_total, snr_db)

    rx_groups = []
    channel_groups = []
    for g in range(groups):
        rx_groups.append(rx_time_total[g*group_size:(g+1)*group_size])
        channel_groups.append(channel_total[g*group_size:(g+1)*group_size])

    return rx_groups, channel_groups

def test_ber(model, lut, N_total, groups, alpha_test, snr_test,
            X_mean, X_std, alpha_train_range, snr_train_range,
            num_symbols=1000, channel_type='rayleigh'):
    """Test BER"""
    group_size = N_total // groups
    bits_per_group = int(np.log2(len(lut)))

    alpha_low, alpha_high = alpha_train_range
    snr_low, snr_high = snr_train_range

    total_bit_errors = 0
    total_bits = 0

    channel_type_value = 0.0 if channel_type == 'awgn' else 1.0

    for i in simple_progress(range(num_symbols),
                            desc=f"  α={alpha_test:.2f}, SNR={snr_test}dB",
                            leave=False):
        true_labels = [np.random.randint(0, len(lut)) for _ in range(groups)]

        rx_groups, channel_groups = transmit_receive_grouped_system(
            true_labels, lut, N_total, groups, alpha_test, snr_test,
            channel_type, 'flat'
        )

        for g in range(groups):
            rx_eq = rx_groups[g] / (channel_groups[g] + 1e-10)
            X_test = np.stack([np.real(rx_eq), np.imag(rx_eq)], axis=-1)
            X_test = X_test.reshape(1, group_size, 2)
            X_test = (X_test - X_mean) / (X_std + 1e-8)

            alpha_norm = np.array([[(alpha_test - alpha_low) / (alpha_high - alpha_low)]])
            snr_norm = np.array([[(snr_test - snr_low) / (snr_high - snr_low)]])
            channel_type_norm = np.array([[channel_type_value]])

            pred = model.predict([X_test, alpha_norm, snr_norm, channel_type_norm], verbose=0)
            detected_label = np.argmax(pred[0])

            true_bits = np.array([int(b) for b in format(true_labels[g], f'0{bits_per_group}b')])
            det_bits = np.array([int(b) for b in format(detected_label, f'0{bits_per_group}b')])
            total_bit_errors += np.sum(true_bits != det_bits)
            total_bits += bits_per_group

    ber = total_bit_errors / total_bits if total_bits > 0 else 0
    return max(ber, 1e-7)

# ============ MAIN TESTING SCRIPT ============

# Load saved files
print("\n>>> Loading saved files...")

# Load LUT
lut_file = os.path.join(SAVED_FILES_PATH, "lut.pkl")
with open(lut_file, 'rb') as f:
    lut_data = pickle.load(f)
    lut = lut_data['lut']
    N = lut_data['N']
    M = lut_data['M']
    N_TOTAL = lut_data['N_TOTAL']
    GROUPS = lut_data['GROUPS']
    num_classes = lut_data['num_classes']

print(f"✓ LUT loaded: {len(lut)} entries, N={N}, M={M}, Groups={GROUPS}")

# Load normalization statistics
stats_file = os.path.join(SAVED_FILES_PATH, "stats.pkl")
with open(stats_file, 'rb') as f:
    stats = pickle.load(f)
    X_mean = stats['X_mean']
    X_std = stats['X_std']
    alpha_low = stats['alpha_low']
    alpha_high = stats['alpha_high']
    snr_low = stats['snr_low']
    snr_high = stats['snr_high']

print(f"✓ Stats loaded: α∈[{alpha_low}, {alpha_high}], SNR∈[{snr_low}, {snr_high}]")

# Build and load model
print("\n>>> Building model architecture...")
model = build_advanced_cnn_transformer_hybrid(
    (N, 2), num_classes, d_model=64, num_heads=4, num_layers=2,
    ff_dim=128, dropout=0.15
)

weights_file = os.path.join(SAVED_FILES_PATH, "phase2_weights.h5")
model.load_weights(weights_file)
print(f"✓ Model loaded from: {weights_file}")
print(f"  Parameters: {model.count_params():,}")

# Run BER testing
print(f"\n{'='*70}")
print("RUNNING BER TESTS")
print(f"{'='*70}")

ber_grid = np.zeros((len(ALPHA_TEST_VALUES), len(SNR_TEST_VALUES)))

for i, alpha_test in enumerate(ALPHA_TEST_VALUES):
    print(f"\nTesting Alpha = {alpha_test:.2f}")
    for j, snr_test in enumerate(SNR_TEST_VALUES):
        ber = test_ber(
            model, lut, N_TOTAL, GROUPS, alpha_test, snr_test,
            X_mean, X_std,
            (alpha_low, alpha_high),
            (snr_low, snr_high),
            num_symbols=NUM_TEST_SYMBOLS,
            channel_type=CHANNEL_TYPE
        )
        ber_grid[i, j] = ber
        print(f"  SNR={snr_test:2d} dB: BER={ber:.6e}")

# Plot results
print(f"\n{'='*70}")
print("GENERATING PLOT")
print(f"{'='*70}")

plt.figure(figsize=(14, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(ALPHA_TEST_VALUES)))

for i, alpha_val in enumerate(ALPHA_TEST_VALUES):
    label = f'α={alpha_val}' + (' (OFDM)' if alpha_val == 1.0 else '')
    plt.semilogy(SNR_TEST_VALUES, ber_grid[i, :],
                marker='o', linewidth=2.5, markersize=7,
                color=colors[i], label=label)

plt.xlabel('SNR (dB)', fontsize=14, fontweight='bold')
plt.ylabel('BER', fontsize=14, fontweight='bold')
plt.title(f'MM-SEFDM: Test Results\n'
          f'N={N}, {num_classes} classes, Groups={GROUPS}\n'
          f'Channel: {CHANNEL_TYPE.upper()}',
          fontsize=12, fontweight='bold')
plt.grid(True, which='both', alpha=0.3)
plt.legend(loc='best', fontsize=11)
plt.ylim([1e-5, 1])
plt.tight_layout()

if SAVE_PLOT:
    plot_path = os.path.join(SAVED_FILES_PATH, PLOT_FILENAME)
    plt.savefig(plot_path, dpi=300)
    print(f"✓ Plot saved to: {plot_path}")

plt.show()

print(f"\n{'='*70}")
print("✓ TESTING COMPLETE!")
print(f"{'='*70}")
print(f"\nBER Results Summary:")
print(f"  Best BER: {np.min(ber_grid):.6e}")
print(f"  Worst BER: {np.max(ber_grid):.6e}")
print(f"{'='*70}")

MM-SEFDM: STANDALONE TESTING
Loading files from: ./MM_SEFDM_GitHub
Test Parameters:
  Alpha values: [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
  SNR range: 0 to 35 dB
  Symbols per test: 1000
  Channel: rayleigh

>>> Loading saved files...
✓ LUT loaded: 256 entries, N=4, M=2, Groups=4
✓ Stats loaded: α∈[0.5, 1.0], SNR∈[0, 35]

>>> Building model architecture...
✓ Model loaded from: ./MM_SEFDM_GitHub\phase2_weights.h5
  Parameters: 272,976

RUNNING BER TESTS

Testing Alpha = 0.50
  α=0.50, SNR=0dB: 1000/1000  SNR= 0 dB: BER=3.510313e-01
  α=0.50, SNR=5dB: 1000/1000  SNR= 5 dB: BER=2.315937e-01
  α=0.50, SNR=10dB: 1000/1000  SNR=10 dB: BER=1.090312e-01
  α=0.50, SNR=15dB: 1000/1000  SNR=15 dB: BER=4.278125e-02
  α=0.50, SNR=20dB: 1000/1000  SNR=20 dB: BER=1.643750e-02
  α=0.50, SNR=25dB: 1000/1000  SNR=25 dB: BER=5.031250e-03
  α=0.50, SNR=30dB: 1000/1000  SNR=30 dB: BER=1.718750e-03
  α=0.50, SNR=35dB: 1000/1000  SNR=35 dB: BER=4.062500e-04

Testing Alpha = 0.60
  α=0.60, SNR=0dB: 200/1000

KeyboardInterrupt: 